# Rankine Cycle
---
### Defines parameters of a Rankine cycle, then returns properties at different states.
___

##### Set the isentropic efficiency of the pump and turbine as well as the maximum pressure in the cycle

In [1]:
import cantera as ct

# parameters
eta_pump = 0.6     # pump isentropic efficiency
eta_turbine = 0.8  # turbine isentropic efficiency
p_max = 8.0e5       # maximum pressure

##### Define adiabatic compression resembling a pump and adiabatic expansion resembling a condenser. These are dependant upon the efficiencies set above

In [2]:
def pump(fluid, p_final, eta):
    """Adiabatically pump a fluid to pressure p_final, using
    a pump with isentropic efficiency eta."""
    h0 = fluid.h
    s0 = fluid.s
    fluid.SP = s0, p_final
    h1s = fluid.h
    isentropic_work = h1s - h0
    actual_work = isentropic_work / eta
    h1 = h0 + actual_work
    fluid.HP = h1, p_final
    return actual_work


def expand(fluid, p_final, eta):
    """Adiabatically expand a fluid to pressure p_final, using
    a turbine with isentropic efficiency eta."""
    h0 = fluid.h
    s0 = fluid.s
    fluid.SP =s0, p_final
    h1s = fluid.h
    isentropic_work = h0 - h1s
    actual_work = isentropic_work * eta
    h1 = h0 - actual_work
    fluid.HP = h1, p_final
    return actual_work

##### Define printState function with parameters of (state, species). 

In [3]:
def printState(n, fluid):
    print('\n***************** State {0} ******************'.format(n))
    print(fluid.report())

##### Create an object that resembles water for the Rankine cycle

In [4]:
if __name__ == '__main__':
    w = ct.Water()

###### Set state 1 with quality of 0 (saturated liquid) and temperature of 300 K

In [5]:
    w.TX = 300.0, 0.0
    h1 = w.h
    p1 = w.P
    printState(1, w)


***************** State 1 ******************

  water:

       temperature             300  K
          pressure         3528.21  Pa
           density         996.589  kg/m^3
  mean mol. weight          18.016  amu
    vapor fraction               0

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy    -1.58582e+07       -2.857e+08     J
   internal energy    -1.58582e+07       -2.857e+08     J
           entropy          3913.2         7.05e+04     J/K
    Gibbs function    -1.70322e+07       -3.069e+08     J
 heat capacity c_p     8.12688e+07        1.464e+09     J/K
 heat capacity c_v         4131.03        7.442e+04     J/K



###### Progress to state 2 through adiabatic compression, determined by p_max. Also calculate work done on pump

In [6]:
    pump_work = pump(w, p_max, eta_pump)
    h2 = w.h
    printState(2, w)


***************** State 2 ******************

  water:

       temperature         300.143  K
          pressure          800000  Pa
           density          996.91  kg/m^3
  mean mol. weight          18.016  amu
    vapor fraction               0

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy    -1.58569e+07       -2.857e+08     J
   internal energy    -1.58577e+07       -2.857e+08     J
           entropy         3914.98        7.053e+04     J/K
    Gibbs function    -1.70319e+07       -3.068e+08     J
 heat capacity c_p          4178.6        7.528e+04     J/K
 heat capacity c_v         4127.86        7.437e+04     J/K



###### Progress to state 3 through isobaric expansion, determined by quality of 1.0 and p_max

In [7]:
    w.PX = p_max, 1.0
    h3 = w.h
    heat_added = h3 - h2
    printState(3, w)


***************** State 3 ******************

  water:

       temperature         443.624  K
          pressure          800000  Pa
           density         4.15875  kg/m^3
  mean mol. weight          18.016  amu
    vapor fraction               1

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy    -1.32016e+07       -2.378e+08     J
   internal energy    -1.33939e+07       -2.413e+08     J
           entropy         10182.9        1.835e+05     J/K
    Gibbs function     -1.7719e+07       -3.192e+08     J
 heat capacity c_p     4.61668e+07        8.317e+08     J/K
 heat capacity c_v         1764.53        3.179e+04     J/K



###### Progress to state 4 through adiabatic expansion. Also determines work done by turbine

In [8]:
    turbine_work = expand(w, p1, eta_turbine)
    printState(4, w)


***************** State 4 ******************

  water:

       temperature             300  K
          pressure         3528.21  Pa
           density       0.0305583  kg/m^3
  mean mol. weight          18.016  amu
    vapor fraction        0.835158

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy    -1.38221e+07        -2.49e+08     J
   internal energy    -1.39376e+07       -2.511e+08     J
           entropy         10700.1        1.928e+05     J/K
    Gibbs function    -1.70322e+07       -3.069e+08     J
 heat capacity c_p    inf              inf              J/K
 heat capacity c_v          108455        1.954e+06     J/K



##### Based on results from the cycle, calculate and print efficiency of the cycle

In [9]:
    eff = (turbine_work - pump_work)/heat_added

    print('efficiency = ', eff)

efficiency =  0.23320858676798334
